# Real or Fake Image Recognition Task

In [1]:
import os
import zipfile
import numpy as np
import random
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image

seed = 42
os.environ["PYTHONHASHSEED"] = str(seed)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

## Load data

In [2]:
class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        for label, sub_dir in enumerate(os.listdir(root_dir)):  # cifar has label 0, uvit has label 1
            full_dir = os.path.join(root_dir, sub_dir)
            for img_name in os.listdir(full_dir):
                img_path = os.path.join(full_dir, img_name)
                self.image_paths.append(img_path)
                self.labels.append(label)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

In [3]:
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # These values are used when grading
])

ds_train = CustomDataset("./train_v1", transform=transform)
dl_train = DataLoader(ds_train, batch_size=64, shuffle=True)

## Define model and train

In [4]:
class MyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 8, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(8, 10, 5)
        self.fc1 = nn.Linear(10 * 5 * 5, 70)
        self.fc2 = nn.Linear(70, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 10 * 5 * 5)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x

In [5]:
def train(model, device, optimizer, criterion, dataloader, num_epochs):
    train_losses = []
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0
        
        for images, labels in dataloader:
            images, labels = images.to(device), labels.float().to(device)
            
            outputs = model(images).squeeze()
            loss = criterion(outputs, labels)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item() * images.size(0)

        running_loss /= len(dataloader.dataset)
        train_losses.append(running_loss)
        accuracy = eval(model, device, dataloader)
        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss}, Accuracy: {accuracy}")
    return train_losses

In [6]:
@torch.no_grad()
def eval(model, device, dataloader):
    model.eval()
    correct = 0
    total = 0
    for images, labels in dataloader:
        images, labels = images.to(device), labels.to(device).float()
        
        outputs = model(images).squeeze()
        preds = outputs >= 0.5
        
        correct += (preds == labels).sum().item()
        total += labels.size(0)
    return correct / total

In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = MyModel().to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [8]:
train(model, device, optimizer, criterion, dl_train, 10);

Epoch [1/10], Loss: 0.6295432175636292, Accuracy: 0.6996
Epoch [2/10], Loss: 0.562065036392212, Accuracy: 0.724
Epoch [3/10], Loss: 0.5570123519897461, Accuracy: 0.7208
Epoch [4/10], Loss: 0.5448735479354858, Accuracy: 0.7262
Epoch [5/10], Loss: 0.5409635952949524, Accuracy: 0.7293
Epoch [6/10], Loss: 0.5313480290412903, Accuracy: 0.7291
Epoch [7/10], Loss: 0.5261241621017456, Accuracy: 0.7508
Epoch [8/10], Loss: 0.5255340951442719, Accuracy: 0.7396
Epoch [9/10], Loss: 0.522794758605957, Accuracy: 0.7531
Epoch [10/10], Loss: 0.5084276931762696, Accuracy: 0.7544


## Save for submission

In [9]:
py_filename = "submission_model.py"
pth_filename = "submission_dic.pth"
zip_filename = "submission.zip"  # Will submit this zip to the grader

In [10]:
torch.save(model.state_dict(), pth_filename)

In [11]:
model_code = """  
import torch
import torch.nn as nn


class MyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 8, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(8, 10, 5)
        self.fc1 = nn.Linear(10 * 5 * 5, 70)
        self.fc2 = nn.Linear(70, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 10 * 5 * 5)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x
""".lstrip()

with open(py_filename, "w") as f:
    f.write(model_code)

In [12]:
with zipfile.ZipFile(zip_filename, "w") as zipf:
    for file in [py_filename, pth_filename]:
        zipf.write(file, os.path.basename(file))

## Score

Leaderboard A accuracy: 1.0000